In [7]:
import sys
sys.path.append('../')
import attention
import jax.numpy as jnp
import numpy as np

In [8]:
from torch.nn import MultiheadAttention
import torch

attn = MultiheadAttention(embed_dim=512, num_heads=1, bias=False)

# print weights of attn
print(attn.in_proj_weight.shape)
print(attn.in_proj_weight.shape[0]/3)

# Torch uses a concatenated weight matrix of size (embed_dim * 3, embed_dim)
x = torch.randn(1, 512)

W_q = torch.randn(512, 512)
W_k = torch.randn(512, 512)
W_v = torch.randn(512, 512)

W = torch.cat([W_q, W_k, W_v], dim=0)
attn.in_proj_weight = torch.nn.Parameter(W)

out, _ = attn(x, x, x)

# print all learnable parameters

for name, param in attn.named_parameters():
    if param.requires_grad:
        print(name, param.data.shape)

torch.Size([1536, 512])
512.0
in_proj_weight torch.Size([1536, 512])
out_proj.weight torch.Size([512, 512])


In [9]:
from attention import *

x = jnp.array(torch.randn(10, 2))
jnp.sum(x, axis=0)

Array([5.091933, 4.222148], dtype=float32)

In [10]:
import math
from typing import Optional, List

import torch
from torch import nn

# from labml import tracker


class PrepareForMultiHeadAttention(nn.Module):
    """
    <a id="PrepareMHA"></a>

    ## Prepare for multi-head attention

    This module does a linear transformation and splits the vector into given
    number of heads for multi-head attention.
    This is used to transform **key**, **query**, and **value** vectors.
    """

    def __init__(self, d_model: int, heads: int, d_k: int, bias: bool):
        super().__init__()
        # Linear layer for linear transform
        self.linear = nn.Linear(d_model, heads * d_k, bias=bias)
        # Number of heads
        self.heads = heads
        # Number of dimensions in vectors in each head
        self.d_k = d_k

    def forward(self, x: torch.Tensor):
        # Input has shape `[seq_len, batch_size, d_model]` or `[batch_size, d_model]`.
        # We apply the linear transformation to the last dimension and split that into
        # the heads.
        print(f"x.shape: {x.shape}")
        head_shape = x.shape[:-1]
        print(f"head_shape: {head_shape}")

        # Linear transform
        x = self.linear(x)
        print(f"linear(x).shape: {x.shape}")

        # Split last dimension into heads
        x = x.view(*head_shape, self.heads, self.d_k)
        print(f"output.shape: {x.shape}")

        # Output has shape `[seq_len, batch_size, heads, d_k]` or `[batch_size, heads, d_model]`
        return x

In [11]:
d_model = 1 # embedding dimension, split among all heads
n_heads = 1
d_k = 1 
seq_len = 10
batch_size = 2

# q, k, v have shape (seq_len, batch_size, d_model) = (context_len, batch_size, emb_size)

forward1 = PrepareForMultiHeadAttention(d_model, n_heads, d_k, bias=False)

x = torch.randn(seq_len, d_model, requires_grad=False)
y_torch = forward1(x);

x.shape: torch.Size([10, 1])
head_shape: torch.Size([10])
linear(x).shape: torch.Size([10, 1])
output.shape: torch.Size([10, 1, 1])


In [12]:
from attention import PreAttention

preattn = PreAttention(emb_size=d_model, n_heads=n_heads, d_k=d_k, bias=False)
xjnp = jnp.array(x)
W = jnp.array(forward1.linear.weight.detach())
state = DenseState(W, None)
print(f"W.shape: {W.shape}")
y_jax = preattn(state, xjnp);


np.allclose(y_torch.detach().numpy(), y_jax, atol=1e-6)

W.shape: (1, 1)
x.shape: (10, 1)
head_shape: (10,)
linear(x).shape: (10, 1)
reshape(x).shape: (10, 1, 1)


True

In [7]:

class MultiHeadAttention(nn.Module):
    def __init__(self, heads: int, d_model: int, dropout_prob: float = 0.1, bias: bool = True):
        """
        * `heads` is the number of heads.
        * `d_model` is the number of features in the `query`, `key` and `value` vectors.
        """

        super().__init__()

        # Number of features per head
        self.d_k = d_model // heads
        # Number of heads
        self.heads = heads

        # These transform the `query`, `key` and `value` vectors for multi-headed attention.
        self.query = PrepareForMultiHeadAttention(d_model, heads, self.d_k, bias=bias)
        self.key = PrepareForMultiHeadAttention(d_model, heads, self.d_k, bias=bias)
        self.value = PrepareForMultiHeadAttention(d_model, heads, self.d_k, bias=True)

        # Softmax for attention along the time dimension of `key`
        self.softmax = nn.Softmax(dim=1)

        # Output layer
        self.output = nn.Linear(d_model, d_model)
        # Dropout
        self.dropout = nn.Dropout(dropout_prob)
        # Scaling factor before the softmax
        self.scale = 1 / math.sqrt(self.d_k)

        # We store attentions so that it can be used for logging, or other computations if needed
        self.attn = None

    def get_scores(self, query: torch.Tensor, key: torch.Tensor):

        # Calculate $Q K^\top$ or $S_{ijbh} = \sum_d Q_{ibhd} K_{jbhd}$
        return torch.einsum('ibhd,jbhd->ijbh', query, key)

    def prepare_mask(self, mask: torch.Tensor, query_shape: List[int], key_shape: List[int]):
        """
        `mask` has shape `[seq_len_q, seq_len_k, batch_size]`, where first dimension is the query dimension.
        If the query dimension is equal to $1$ it will be broadcasted.
        """

        assert mask.shape[0] == 1 or mask.shape[0] == query_shape[0]
        assert mask.shape[1] == key_shape[0]
        assert mask.shape[2] == 1 or mask.shape[2] == query_shape[1]

        # Same mask applied to all heads.
        mask = mask.unsqueeze(-1)

        # resulting mask has shape `[seq_len_q, seq_len_k, batch_size, heads]`
        return mask

    def forward(self, *,
                query: torch.Tensor,
                key: torch.Tensor,
                value: torch.Tensor,
                mask: Optional[torch.Tensor] = None):
        """
        `query`, `key` and `value` are the tensors that store
        collection of *query*, *key* and *value* vectors.
        They have shape `[seq_len, batch_size, d_model]`.

        `mask` has shape `[seq_len, seq_len, batch_size]` and
        `mask[i, j, b]` indicates whether for batch `b`,
        query at position `i` has access to key-value at position `j`.
        """

        # `query`, `key` and `value`  have shape `[seq_len, batch_size, d_model]`
        seq_len, batch_size, _ = query.shape

        if mask is not None:
            mask = self.prepare_mask(mask, query.shape, key.shape)

        # Prepare `query`, `key` and `value` for attention computation.
        # These will then have shape `[seq_len, batch_size, heads, d_k]`.
        query = self.query(query)
        key = self.key(key)
        value = self.value(value)

        # Compute attention scores $Q K^\top$.
        # This gives a tensor of shape `[seq_len, seq_len, batch_size, heads]`.
        scores = self.get_scores(query, key)

        # Scale scores $\frac{Q K^\top}{\sqrt{d_k}}$
        scores *= self.scale

        # Apply mask
        if mask is not None:
            scores = scores.masked_fill(mask == 0, float('-inf'))

        # $softmax$ attention along the key sequence dimension
        # $\underset{seq}{softmax}\Bigg(\frac{Q K^\top}{\sqrt{d_k}}\Bigg)$
        attn = self.softmax(scores)

        # Save attentions if debugging
        #tracker.debug('attn', attn)

        # Apply dropout
        attn = self.dropout(attn)

        # Multiply by values
        # $$\underset{seq}{softmax}\Bigg(\frac{Q K^\top}{\sqrt{d_k}}\Bigg)V$$
        x = torch.einsum("ijbh,jbhd->ibhd", attn, value)

        # Save attentions for any other calculations 
        self.attn = attn.detach()

        # Concatenate multiple heads
        x = x.reshape(seq_len, batch_size, -1)

        # Output layer
        return self.output(x)